## 日英翻訳によるデータ拡張

- JParaCrawl big model v3.0 を使用
- パスは適当に設定してください

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ライブラリのインストール
! pip install sentencepiece
! pip install fairseq==0.12.1
! pip install sacrebleu
! pip install sacrebleu[ja]

# テストセット(test,dev)であるwmt20のロード
! sacrebleu -t wmt20 -l en-ja --echo src > wmt.test.en
! sacrebleu -t wmt20 -l en-ja --echo ref > wmt.test.ja
! sacrebleu -t wmt20/dev -l en-ja --echo src > wmt.valid.en
! sacrebleu -t wmt20/dev -l en-ja --echo ref > wmt.valid.ja

In [ ]:
import re
import unicodedata
import sentencepiece as spm

### 前処理



*   NFKC正規化
*   分長比率の高い物の削除



In [ ]:
def remove_spm(text,mode='ja'):
  text = text.replace('▁','')
  if mode == 'ja':
    text = text.replace(' ','')
  # NFKC正規化(半角全角の正規化)
  return unicodedata.normalize("NFKC",text)

en = "Shin  bashi ' s Nine teen th F if teen th F if teen"
print(remove_spm(en,'en'))
ja = "▁この 頃 の 郡 司 正 勝 からの 助言 が 、 その後 の 道を 決めた 。"
remove_spm(ja)

In [ ]:
"""# 比率調査
日英翻訳の際に、質の悪いものを弾く
# 5.0より大きいものまたはまたは0.2より小さいを弾く
結果：635の除去
例１
6.0
Д  а  с  т  и  я  д  е  л  ь  д  е  л  ь  д  е  л  ь
 ハイウエストデニム
別の言語が出力されてる例

例２
0.18181818181818182
News - Yahoo ! JAPAN News
 7日(日)も日差しありただ関東や東海など太平洋側を中心に所々で雨雲発生(tenki.jp)-Yahoo!ニュース
部分的にしか翻訳されていない例
"""

MAX_RATE = 5
MIN_RATE = 0.2

spj = spm.SentencePieceProcessor()
spj.Load("/content/MyDrive/enja_spm_models/spm.ja.nopretok.model")
spe = spm.SentencePieceProcessor()
spe.Load("/content/MyDrive/enja_spm_models/spm.en.nopretok.model")

num = 0
with open('/content/drive/MyDrive/ja-news/news-crawl21-500k.ja') as f1,open('/content/drive/MyDrive/ja-news/news-crawl21-500k.en') as f2,\
open('/content/drive/MyDrive/ja-news/news-crawl21-500k-rem-nfkc.ja','w') as fw1,open('/content/drive/MyDrive/ja-news/news-crawl21-500k-rem-nfkc.en','w') as fw2:
  for ja,en in zip(f1,f2):
    rate = len(en.split(' ')) / len(ja.split(' '))
    if rate > MAX_RATE or rate < MIN_RATE:
      print(rate)
      ja = remove_spm(ja,'ja')
      en = remove_spm(en,'en')
      print(ja,en)
      num += 1
    else:
      ja = remove_spm(ja,'ja')
      en = remove_spm(en,'en')
      fw1.write(" ".join(spj.EncodeAsPieces(ja)) + "\n")
      fw2.write(" ".join(spe.EncodeAsPieces(en)) + "\n")
  print(num) # 対象文書対の数

### 学習とデータ数調整

In [ ]:
! Google DriveのColab Notebooksフォルダへの移動
%cd "drive/MyDrive"

# 各データのDL
# DLしていなかったら、してください
"""!wget 'https://www.kecl.ntt.co.jp/icl/lirg/jparacrawl/release/1.0/pretrained_models/en-ja/big.tar.gz'
!tar -zxvf big.tar.gz"""
"""!wget 'https://www.kecl.ntt.co.jp/icl/lirg/jparacrawl/release/3.0/spm_models/en-ja_spm.tar.gz'
!tar -zxvf en-ja_spm.tar.gz"""
"""!wget 'https://data.statmt.org/news-crawl/ja/news.2021.ja.shuffled.deduped.gz'
!tar -zxvf news.2021.ja.shuffled.deduped.gz"""